In [1]:
#https://github.com/neolithian/banglanews
!pip install banglanews
!pip install bnunicodenormalizer
!pip install pandarallel

  Preparing metadata (setup.py) ... - done
  Created wheel for bnunicodenormalizer: filename=bnunicodenormalizer-0.1.1-py3-none-any.whl size=21913 sha256=c2551ed136aed61f47a4c501a4cc547d74b129ec179a82636e36e2e074c19a3c
  Stored in directory: /root/.cache/pip/wheels/a2/4f/12/67639dde670a3ccd1edcb84d712bb73eb734f822a365109af4
Successfully built bnunicodenormalizer


In [2]:
!mkdir -p /tmp/prothom_alo_data

!mkdir -p /kaggle/working/bn_clean

In [3]:
#prothom_alo
from banglanews import prothomalo

objScraper = prothomalo.scraper('2022-01-01','2022-10-02','/tmp/prothom_alo_data/')

objScraper

objScraper.PrintContents()

Calculating batch process size...
Estimated total URL found in the given criteria: 67307
Calculating batch process size done.
Process started...
Processing batch job 1 out of 275 : at 09:08:06 AM
URL found in this batch: 214
Processing batch job 2 out of 275 : at 09:08:07 AM
URL found in this batch: 229
Processing batch job 3 out of 275 : at 09:08:09 AM
URL found in this batch: 245
Processing batch job 4 out of 275 : at 09:08:10 AM
URL found in this batch: 246
Processing batch job 5 out of 275 : at 09:08:12 AM
URL found in this batch: 262
Processing batch job 6 out of 275 : at 09:08:13 AM
URL found in this batch: 234
Processing batch job 7 out of 275 : at 09:08:15 AM
URL found in this batch: 213
Processing batch job 8 out of 275 : at 09:08:16 AM
URL found in this batch: 208
Processing batch job 9 out of 275 : at 09:08:18 AM
URL found in this batch: 218
Processing batch job 10 out of 275 : at 09:08:19 AM
URL found in this batch: 247
Processing batch job 11 out of 275 : at 09:08:21 AM
UR

In [4]:
import glob
import os 
import pandas as pd
from tqdm import tqdm
import re
#normalization
from pandarallel import pandarallel
from bnunicodenormalizer import Normalizer 
pandarallel.initialize(progress_bar=False,nb_workers=8)
tqdm.pandas()
bnorm=Normalizer()

# Set environment variables
import warnings
warnings.filterwarnings('ignore')

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [5]:
def remove_punctuations(my_str):
    punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-[]{};:'"“\’…,<>.‚/?@#$%^&*_~‘—॥”‰🤣⚽️✌�￰৷￰'''
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

In [6]:
def normalize(sen):
    _words = [bnorm(word)['normalized']  for word in sen.split()]
    return " ".join([word for word in _words if word is not None]) 

In [7]:
ls = glob.glob('/tmp/prothom_alo_data/*')
ls_td = [tex for tex in ls if len(tex)<20]
if len(ls_td) > 0:
    ls.remove(ls_td[0])

In [8]:
f_name = 1
for txt in tqdm(ls):
    with open(txt) as f:
        lines = f.readlines()
        if (len(lines)) == 6:
            lines = lines[5]
            lines = re.split(r'[,;!?।]', lines)
            if len(lines) > 5:
                df = pd.DataFrame()
                df['sentence'] = lines
                df = df.dropna(how='all')
                df['sentence'] = df['sentence'].apply(lambda x : remove_punctuations(x))
                df["sentence"] = df["sentence"].parallel_apply(lambda x:normalize(x))
                df.to_csv(f'/kaggle/working/bn_clean/{f_name}.csv',index=False)
                f_name += 1

100%|██████████| 67228/67228 [6:07:49<00:00,  3.05it/s]
